In [2]:
# The code was removed by Watson Studio for sharing.

In [2]:
!pip install imbalanced-learn

     |████████████████████████████████| 167 kB 16.0 MB/s eta 0:00:01


In [3]:
import pandas as pd
import numpy as np
import networkx as nx
import pickle

from scipy.sparse import hstack, csr_matrix
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from imblearn.ensemble import RUSBoostClassifier
from sklearn.metrics import recall_score, precision_score, roc_auc_score, roc_curve
from sklearn.preprocessing import MinMaxScaler

import warnings; warnings.simplefilter('ignore')

<h1 style="font-size:10 em">Set Parameters and Import Data</h1>

The parameters T, t1, and t2 below may be chosen by the user.  Extreme choices may affect model performance.

In [4]:
T = 50  #set threshold parameter for "popular recipe"
t1 = pd.Timestamp('2016-12-31') #set initial time parameter upon which to base predictions.  Constraint:  t1 > 01-01-2001 + (t2 - t1)
t2 = pd.Timestamp('2018-03-30') #set prediction time parameter

In [5]:
days = pd.Timedelta((((t2 - t1).days // 365) + 1)*365,unit='D')

m1 = t1 - days # go back to previous time for model-training purposes
m2 = m1 + (t2 - t1)

m1,m2

(Timestamp('2015-01-01 00:00:00'), Timestamp('2016-03-30 00:00:00'))

In [6]:
recipes = pd.read_csv(project.get_file('recipes.csv'),index_col=0)
recipes['submitted'] = pd.to_datetime(recipes['submitted'])

recipes

,name,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
id,,,,,,,,,,,
137739,arriba baked winter squash mexican style,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
31490,a bit different breakfast pizza,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
112140,all in the kitchen chili,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
59389,alouette potatoes,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
44061,amish tomato ketchup for canning,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8
...,...,...,...,...,...,...,...,...,...,...,...
486161,zydeco soup,60,227978,2012-08-29,"['ham', '60-minutes-or-less', 'time-to-make', ...","[415.2, 26.0, 34.0, 26.0, 44.0, 21.0, 15.0]",7,"['heat oil in a 4-quart dutch oven', 'add cele...",this is a delicious soup that i originally fou...,"['celery', 'onion', 'green sweet pepper', 'gar...",22
493372,zydeco spice mix,5,1500678,2013-01-09,"['15-minutes-or-less', 'time-to-make', 'course...","[14.8, 0.0, 2.0, 58.0, 1.0, 0.0, 1.0]",1,['mix all ingredients together thoroughly'],this spice mix will make your taste buds dance!,"['paprika', 'salt', 'garlic powder', 'onion po...",13
308080,zydeco ya ya deviled eggs,40,37779,2008-06-07,"['60-minutes-or-less', 'time-to-make', 'course...","[59.2, 6.0, 2.0, 3.0, 6.0, 5.0, 0.0]",7,"['in a bowl , combine the mashed yolks and may...","deviled eggs, cajun-style","['hard-cooked eggs', 'mayonnaise', 'dijon must...",8


In [7]:
interactions = pd.read_csv(project.get_file('interactions.csv'),index_col=0)
interactions['date'] = pd.to_datetime(interactions['date'])

interactions

,user_id,recipe_id,date,rating,review
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall..."
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin..."
...,...,...,...,...,...
1132362,116593,72730,2003-12-09,0,Another approach is to start making sauce with...
1132363,583662,386618,2009-09-29,5,These were so delicious! My husband and I tru...
1132364,157126,78003,2008-06-23,5,WOW! Sometimes I don't take the time to rate ...
1132365,53932,78003,2009-01-11,4,Very good! I used regular port as well. The ...


<h1 style="font-size:10 em">Feature Engineering:</h1>

<h2>1. Current No. of Ratings</h2>
For the training data the number of ratings is taken at time m1.  For the test data, we take time t1.

<h2>2. Age in Days</h2>
Training data is age at time m1; test data is age at time t1.

<h2>3. Rating Pace</h2>
(current no. of ratings) / (age in days)

In [8]:
interactions_pre_m1 = interactions[interactions['date'] <= m1]
interactions_pre_m2 = interactions[interactions['date'] <= m2]
interactions_pre_t1 = interactions[interactions['date'] <= t1]
interactions_pre_t2 = interactions[interactions['date'] <= t2]

recipe_ratings_count_m1 = (interactions_pre_m1.groupby('recipe_id').agg({'rating':'count'})
    .rename(columns={'rating':'total_ratings_at_m1'}))
recipe_ratings_count_m2 = (interactions_pre_m2.groupby('recipe_id').agg({'rating':'count'})
    .rename(columns={'rating':'total_ratings_at_m2'}))
recipe_ratings_count_t1 = (interactions_pre_t1.groupby('recipe_id').agg({'rating':'count'})
    .rename(columns={'rating':'total_ratings_at_t1'}))
recipe_ratings_count_t2 = (interactions_pre_t2.groupby('recipe_id').agg({'rating':'count'})
    .rename(columns={'rating':'total_ratings_at_t2'}))
recipe_ratings_count = pd.merge(recipe_ratings_count_m1,recipe_ratings_count_m2,how='outer',left_index=True,right_index=True)
recipe_ratings_count = pd.merge(recipe_ratings_count,recipe_ratings_count_t1,how='outer',left_index=True,right_index=True)
recipe_ratings_count = pd.merge(recipe_ratings_count,recipe_ratings_count_t2,how='outer',left_index=True,right_index=True)
recipe_ratings_count['total_ratings_at_m1'] = recipe_ratings_count['total_ratings_at_m1'].fillna(0).astype(int)
recipe_ratings_count['total_ratings_at_m2'] = recipe_ratings_count['total_ratings_at_m2'].fillna(0).astype(int)
recipe_ratings_count['total_ratings_at_t1'] = recipe_ratings_count['total_ratings_at_t1'].fillna(0).astype(int)
recipe_ratings_count['m1_target'] = recipe_ratings_count['total_ratings_at_m1'].map(lambda x: x >= T).astype(int)
recipe_ratings_count['m2_target'] = recipe_ratings_count['total_ratings_at_m2'].map(lambda x: x >= T).astype(int)
recipe_ratings_count['t1_target'] = recipe_ratings_count['total_ratings_at_t1'].map(lambda x: x >= T).astype(int)
recipe_ratings_count['t2_target'] = recipe_ratings_count['total_ratings_at_t2'].map(lambda x: x >= T).astype(int)
recipe_ratings_count

,total_ratings_at_m1,total_ratings_at_m2,total_ratings_at_t1,total_ratings_at_t2,m1_target,m2_target,t1_target,t2_target
recipe_id,,,,,,,,
38,4,4,4,4,0,0,0,0
39,1,1,1,1,0,0,0,0
40,9,9,9,9,0,0,0,0
41,2,2,2,2,0,0,0,0
43,1,1,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...
535233,0,0,0,1,0,0,0,0
535269,0,0,0,1,0,0,0,0
535288,0,0,0,1,0,0,0,0


In [9]:
df_train = (recipe_ratings_count[['m2_target','total_ratings_at_m1']]
            .loc[recipe_ratings_count['m1_target'] == 0]
            .rename(columns={'m2_target':'target','total_ratings_at_m1':'total_ratings_curr'})
           )

df_train = pd.merge(df_train,recipes['submitted'],how='left',left_index=True,right_index=True)
df_train['age_in_days'] = ((m1 - df_train['submitted']) / pd.Timedelta(1,'D')).astype(int)
df_train.drop(columns=['submitted'],inplace=True)
df_train = df_train.loc[df_train['age_in_days'] >= 0]
df_train['rating_pace'] = df_train['total_ratings_curr'] / df_train['age_in_days']

df_train

,target,total_ratings_curr,age_in_days,rating_pace
recipe_id,,,,
38,0,4,5624,0.000711
39,0,1,5604,0.000178
40,0,9,5597,0.001608
41,0,2,5599,0.000357
43,0,1,5612,0.000178
...,...,...,...,...
520245,0,0,3,0.000000
520248,0,1,3,0.333333
520250,0,1,3,0.333333


In [10]:
df_test = (recipe_ratings_count[['t2_target','total_ratings_at_t1']]
            .loc[recipe_ratings_count['t1_target'] == 0]
            .rename(columns={'t2_target':'target','total_ratings_at_t1':'total_ratings_curr'})
           )

df_test = pd.merge(df_test,recipes['submitted'],how='left',left_index=True,right_index=True)
df_test['age_in_days'] = ((t1 - df_test['submitted']) / pd.Timedelta(1,'D')).astype(int)
df_test.drop(columns=['submitted'],inplace=True)
df_test = df_test.loc[df_test['age_in_days'] >= 0]
df_test['rating_pace'] = df_test['total_ratings_curr'] / df_test['age_in_days']

df_test

,target,total_ratings_curr,age_in_days,rating_pace
recipe_id,,,,
38,0,4,6354,0.000630
39,0,1,6334,0.000158
40,0,9,6327,0.001422
41,0,2,6329,0.000316
43,0,1,6342,0.000158
...,...,...,...,...
529564,0,0,11,0.000000
529568,0,0,11,0.000000
529577,0,1,10,0.100000


<h2>4. RUS Boosted Naive Bayes Probability Prediction</h2>

Aggregates of all reviews up to the current moment (time m1 for training data; time t1 for test data) are collected, vectorized, and used to train a RUS (Random Undersampling) Boosted classifier with Multinomial Naive Bayes models as the base estimators.  The classifier returns a natural estimate of the probability that a given recipe will cross the threshold of T ratings at time (m2 for training, t2 for test).

The probability itself should not be taken very literally, but it makes a useful feature which improves our model performance later.

In exploratory data analysis (not included in this document), the RUS NB classifier itself has tended to perform with extremely high recall (> .98 even at low thresholds) but low precision (< .002).  This is not very surprising, because we are trying to predict an extremely rare event.  The classifier is generally able to find them, but makes a lot of false positive guesses in the process.  Still, the RUS NB classifier is able to narrow down the search by eliminating tens of thousands of recipes, which facilitates a more efficient network analysis later in this notebook (see Feature Engineering parts 12 and 13).

In [11]:
inters_pre_m1 = interactions.loc[interactions['date'] <= m1]
inters_pre_m1['review'].fillna('',inplace=True)
review_aggs_m1 = inters_pre_m1.groupby('recipe_id').agg({'review':(lambda x: ' '.join(x))}).rename(columns={'review':'review_agg_at_m1'})
              
inters_pre_t1 = interactions.loc[interactions['date'] <= t1]
inters_pre_t1['review'].fillna('',inplace=True)
review_aggs_t1 = inters_pre_t1.groupby('recipe_id').agg({'review':(lambda x: ' '.join(x))}).rename(columns={'review':'review_agg_at_t1'})

In [12]:
review_aggs_m1

,review_agg_at_m1
recipe_id,
38,"Yummy, yummy, yummy! I am a big fan of fruit a..."
39,I have an Indian friend who made this dish for...
40,My favourite lemonade recipe is essentially th...
41,I thought this was very GOOD! ANYONE who is af...
43,did not work.
...,...
520088,This recipe calls for walnuts and the instruct...
520110,"Yeah I did it! The cake was wonderful, I made ..."
520197,This is one of the best potato soups I have ev...


In [13]:
review_aggs_t1

,review_agg_at_t1
recipe_id,
38,"Yummy, yummy, yummy! I am a big fan of fruit a..."
39,I have an Indian friend who made this dish for...
40,My favourite lemonade recipe is essentially th...
41,I thought this was very GOOD! ANYONE who is af...
43,did not work.
...,...
529309,Juicy Juicy cook it for dinner almost every ni...
529310,AWESOME very sweet and delightful SOOOOOOO good
529313,Awesome Mosit as hell OMG


In [14]:
df_train = pd.merge(df_train,review_aggs_m1,how='left',left_index=True,right_index=True)
df_train['review_agg_at_m1'].fillna('',inplace=True)

df_train

,target,total_ratings_curr,age_in_days,rating_pace,review_agg_at_m1
recipe_id,,,,,
38,0,4,5624,0.000711,"Yummy, yummy, yummy! I am a big fan of fruit a..."
39,0,1,5604,0.000178,I have an Indian friend who made this dish for...
40,0,9,5597,0.001608,My favourite lemonade recipe is essentially th...
41,0,2,5599,0.000357,I thought this was very GOOD! ANYONE who is af...
43,0,1,5612,0.000178,did not work.
...,...,...,...,...,...
520245,0,0,3,0.000000,
520248,0,1,3,0.333333,"This is really good and so easy., I made it fo..."
520250,0,1,3,0.333333,This was a fantastic dish! My family and frien...


In [15]:
vect = CountVectorizer(min_df=3).fit(df_train['review_agg_at_m1'])  #train vectorizer on review aggregates at time m1

df_train_reviews_vectorized = vect.transform(df_train['review_agg_at_m1'])  #transform training review aggregates to sparse matrix

df_train.drop(columns='review_agg_at_m1',inplace=True)  #drop review aggregates from training feature set

RUSNB_X_train = hstack([df_train['total_ratings_curr'].values.reshape(-1,1),df_train_reviews_vectorized])  #build sparse matrix for training RUSNB classifier
RUSNB_y_train = df_train['target'].values.reshape(-1,1)

In [16]:
RUSNB_clfr = RUSBoostClassifier(base_estimator=MultinomialNB(alpha=0.01),sampling_strategy=0.2,random_state=0).fit(RUSNB_X_train, RUSNB_y_train)  #  Train RUSNB classifier. 
                                                                                                         #     Hyperparameters were tuned previously in exploratory phase.
    
predict_probas = RUSNB_clfr.predict_proba(RUSNB_X_train)[:,1] #make probability predictions

fpr, tpr, thresholds = roc_curve(RUSNB_y_train,predict_probas) #determine good threshold for classifier evaluation
thresh = thresholds[np.argmax(tpr - fpr)]

predictions = (predict_probas >= thresh) #make predictions based on good threshold
        
roc_auc_score(RUSNB_y_train,predict_probas), recall_score(RUSNB_y_train,predictions), precision_score(RUSNB_y_train,predictions)  #evaluate on training data

(0.8864089627996476, 1.0, 0.0012332832312020657)

In [17]:
project.save_data('vect_and_RUSNB_clfr.pkl', pickle.dumps((vect, RUSNB_clfr)), set_project_asset=True, overwrite=True)  # save vectorizer and trained RUSNB classifier

{'file_name': 'vect_and_RUSNB_clfr.pkl',
 'message': 'File saved to project storage.',
 'bucket_name': 'foodcomrecipepopularitypredictor-donotdelete-pr-wiqbzxzcs7ksbf',
 'asset_id': 'cbd1ac25-a4b4-4c4f-a527-da7308ca8867'}

In [18]:
df_train['RUSNB_proba_pred'] = predict_probas

df_train

,target,total_ratings_curr,age_in_days,rating_pace,RUSNB_proba_pred
recipe_id,,,,,
38,0,4,5624,0.000711,1.190284e-01
39,0,1,5604,0.000178,1.485670e-02
40,0,9,5597,0.001608,5.000000e-01
41,0,2,5599,0.000357,4.268798e-03
43,0,1,5612,0.000178,3.387240e-08
...,...,...,...,...,...
520245,0,0,3,0.000000,1.303901e-08
520248,0,1,3,0.333333,1.055873e-01
520250,0,1,3,0.333333,3.360625e-06


In [19]:
df_test = pd.merge(df_test,review_aggs_t1,how='left',left_index=True,right_index=True)
df_test['review_agg_at_t1'].fillna('',inplace=True)

df_test

,target,total_ratings_curr,age_in_days,rating_pace,review_agg_at_t1
recipe_id,,,,,
38,0,4,6354,0.000630,"Yummy, yummy, yummy! I am a big fan of fruit a..."
39,0,1,6334,0.000158,I have an Indian friend who made this dish for...
40,0,9,6327,0.001422,My favourite lemonade recipe is essentially th...
41,0,2,6329,0.000316,I thought this was very GOOD! ANYONE who is af...
43,0,1,6342,0.000158,did not work.
...,...,...,...,...,...
529564,0,0,11,0.000000,
529568,0,0,11,0.000000,
529577,0,1,10,0.100000,This is an old Pillsbury bake off winning reci...


In [20]:
df_test_reviews_vectorized = vect.transform(df_test['review_agg_at_t1'])  #transform test review aggregates to sparse matrix

df_test.drop(columns='review_agg_at_t1',inplace=True)  #drop review aggregates from test feature set

RUSNB_X_test = hstack([df_test['total_ratings_curr'].values.reshape(-1,1),df_test_reviews_vectorized])  #build sparse matrix for RUSNB classifier
RUSNB_y_test = df_test['target'].values.reshape(-1,1)

predict_probas_test = RUSNB_clfr.predict_proba(RUSNB_X_test)[:,1] #make probability predictions

predictions_test = (predict_probas_test >= thresh) #make predictions based on good threshold
        
roc_auc_score(RUSNB_y_test,predict_probas_test), recall_score(RUSNB_y_test,predictions_test), precision_score(RUSNB_y_test,predictions_test)  #evaluate on test data

(0.8832465087773059, 1.0, 0.0023006565288143203)

In [21]:
df_test['RUSNB_proba_pred'] = predict_probas_test

df_test

,target,total_ratings_curr,age_in_days,rating_pace,RUSNB_proba_pred
recipe_id,,,,,
38,0,4,6354,0.000630,1.190284e-01
39,0,1,6334,0.000158,1.485670e-02
40,0,9,6327,0.001422,5.000000e-01
41,0,2,6329,0.000316,4.268798e-03
43,0,1,6342,0.000158,3.387240e-08
...,...,...,...,...,...
529564,0,0,11,0.000000,1.303901e-08
529568,0,0,11,0.000000,1.303901e-08
529577,0,1,10,0.100000,2.218845e-06


<h2>5. Current Mean Rating</h2>
Training data is mean rating at time m1; test data is mean rating at time t1.

<h2>6. Number of Minutes</h2>

<h2>7. Number of Steps</h2>

<h2>8. Number of Ingredients</h2>

These three features are provided directly in the raw data set.

<h2>9. Number of Calories</h2>

This feature is extracted from text data (which was previously aggregated by the original publishers of the Food.com data set).

In [22]:
mean_rating_at_m1 = inters_pre_m1.groupby('recipe_id').mean().rename(columns={'rating':'mean_rating_at_m1'})

df_train = pd.merge(df_train,mean_rating_at_m1['mean_rating_at_m1'],how='left',left_index=True,right_index=True)
df_train.rename(columns={'mean_rating_at_m1':'mean_rating_curr'},inplace=True)
df_train['mean_rating_curr'].fillna(0,inplace=True)

df_train = pd.merge(df_train,recipes[['minutes','n_steps','n_ingredients','nutrition']],how='left',left_index=True,right_index=True)
df_train['calories'] = df_train['nutrition'].str.extract(r'[[]([^,]*)')
df_train.drop(columns='nutrition',inplace=True)

df_train

,target,total_ratings_curr,age_in_days,rating_pace,RUSNB_proba_pred,mean_rating_curr,minutes,n_steps,n_ingredients,calories
recipe_id,,,,,,,,,,
38,0,4,5624,0.000711,1.190284e-01,4.250000,1485,13,4,170.9
39,0,1,5604,0.000178,1.485670e-02,3.000000,265,17,26,1110.7
40,0,9,5597,0.001608,5.000000e-01,4.333333,35,8,6,311.1
41,0,2,5599,0.000357,4.268798e-03,4.500000,1460,18,15,536.1
43,0,1,5612,0.000178,3.387240e-08,1.000000,140,35,15,437.9
...,...,...,...,...,...,...,...,...,...,...
520245,0,0,3,0.000000,1.303901e-08,0.000000,35,10,6,206.6
520248,0,1,3,0.333333,1.055873e-01,5.000000,15,9,7,473.7
520250,0,1,3,0.333333,3.360625e-06,5.000000,30,11,12,251.5


In [23]:
mean_rating_at_t1 = inters_pre_t1.groupby('recipe_id').mean().rename(columns={'rating':'mean_rating_at_t1'})

df_test = pd.merge(df_test,mean_rating_at_t1['mean_rating_at_t1'],how='left',left_index=True,right_index=True)
df_test.rename(columns={'mean_rating_at_t1':'mean_rating_curr'},inplace=True)
df_test['mean_rating_curr'].fillna(0,inplace=True)

df_test = pd.merge(df_test,recipes[['minutes','n_steps','n_ingredients','nutrition']],how='left',left_index=True,right_index=True)
df_test['calories'] = df_test['nutrition'].str.extract(r'[[]([^,]*)')
df_test.drop(columns='nutrition',inplace=True)

df_test

,target,total_ratings_curr,age_in_days,rating_pace,RUSNB_proba_pred,mean_rating_curr,minutes,n_steps,n_ingredients,calories
recipe_id,,,,,,,,,,
38,0,4,6354,0.000630,1.190284e-01,4.250000,1485,13,4,170.9
39,0,1,6334,0.000158,1.485670e-02,3.000000,265,17,26,1110.7
40,0,9,6327,0.001422,5.000000e-01,4.333333,35,8,6,311.1
41,0,2,6329,0.000316,4.268798e-03,4.500000,1460,18,15,536.1
43,0,1,6342,0.000158,3.387240e-08,1.000000,140,35,15,437.9
...,...,...,...,...,...,...,...,...,...,...
529564,0,0,11,0.000000,1.303901e-08,0.000000,55,8,9,908.9
529568,0,0,11,0.000000,1.303901e-08,0.000000,35,21,12,358.1
529577,0,1,10,0.100000,2.218845e-06,0.000000,25,12,5,132.5


<h2>10. Instructions Readability Modified LIX Score</h2>

For each recipe, the readability score is computed from the text of the recipe instructions, as follows:

100 * (number of long words) / (number of words) + (number of words) / (number of recipe steps)

where words are considered long if they contain more than 6 characters.  This is the same as the standard LIX score except we are using number of recipe steps instead of number of sentences.

In [24]:
simple_word_tokenizer = RegexpTokenizer(r'\w+')

readability_df = recipes['steps'].to_frame().fillna('')
readability_df['steps_word_tokenized'] = readability_df['steps'].apply(simple_word_tokenizer.tokenize)
readability_df['long_words'] = readability_df['steps_word_tokenized'].apply(lambda x: [word for word in x if len(word) > 6])

readability_df = pd.merge(readability_df, recipes['n_steps'],how='left',left_index=True,right_index=True)
readability_df['num_words'] = readability_df['steps_word_tokenized'].apply(lambda x: len(x))
readability_df['num_long_words'] = readability_df['long_words'].apply(lambda x: len(x))

readability_df['LIX_score'] = 100 * readability_df['num_long_words'] / readability_df['num_words'] + readability_df['num_words'] / readability_df['n_steps']

In [25]:
df_train = pd.merge(df_train,readability_df['LIX_score'],how='left',left_index=True,right_index=True)
df_train['LIX_score'].fillna(0,inplace=True)

df_train

,target,total_ratings_curr,age_in_days,rating_pace,RUSNB_proba_pred,mean_rating_curr,minutes,n_steps,n_ingredients,calories,LIX_score
recipe_id,,,,,,,,,,,
38,0,4,5624,0.000711,1.190284e-01,4.250000,1485,13,4,170.9,28.360140
39,0,1,5604,0.000178,1.485670e-02,3.000000,265,17,26,1110.7,33.459893
40,0,9,5597,0.001608,5.000000e-01,4.333333,35,8,6,311.1,15.944444
41,0,2,5599,0.000357,4.268798e-03,4.500000,1460,18,15,536.1,31.488889
43,0,1,5612,0.000178,3.387240e-08,1.000000,140,35,15,437.9,28.777387
...,...,...,...,...,...,...,...,...,...,...,...
520245,0,0,3,0.000000,1.303901e-08,0.000000,35,10,6,206.6,31.076623
520248,0,1,3,0.333333,1.055873e-01,5.000000,15,9,7,473.7,30.761905
520250,0,1,3,0.333333,3.360625e-06,5.000000,30,11,12,251.5,31.080946


In [26]:
df_test = pd.merge(df_test,readability_df['LIX_score'],how='left',left_index=True,right_index=True)
df_test['LIX_score'].fillna(0,inplace=True)

df_test

,target,total_ratings_curr,age_in_days,rating_pace,RUSNB_proba_pred,mean_rating_curr,minutes,n_steps,n_ingredients,calories,LIX_score
recipe_id,,,,,,,,,,,
38,0,4,6354,0.000630,1.190284e-01,4.250000,1485,13,4,170.9,28.360140
39,0,1,6334,0.000158,1.485670e-02,3.000000,265,17,26,1110.7,33.459893
40,0,9,6327,0.001422,5.000000e-01,4.333333,35,8,6,311.1,15.944444
41,0,2,6329,0.000316,4.268798e-03,4.500000,1460,18,15,536.1,31.488889
43,0,1,6342,0.000158,3.387240e-08,1.000000,140,35,15,437.9,28.777387
...,...,...,...,...,...,...,...,...,...,...,...
529564,0,0,11,0.000000,1.303901e-08,0.000000,55,8,9,908.9,30.330645
529568,0,0,11,0.000000,1.303901e-08,0.000000,35,21,12,358.1,26.595238
529577,0,1,10,0.100000,2.218845e-06,0.000000,25,12,5,132.5,33.365248


<h2>11. Recipe Innovation IDF</h2>

We first look at each ingredient appearing in some recipe by the current time (m1 for training data, t1 for test data).  Ingredients appearing in fewer than 50 recipes are dropped.  The remaining ingredients are assigned a rareness score: 

(number of recipes containing the ingredient) / (total number of recipes)

Then the innovation IDF of a recipe is computed as the average rareness score across each of its ingredients.  A high innovation IDF score indicates a recipe with many common ingredients; and a low score a recipe with rare ingredients.

A small percentage of recipes do not contain any common ingredients, which can lead to NaNs in the computations below.  These are filled in with 0's.

In [28]:
infile = project.get_file("ingr_map.pkl")
ingr_map = pickle.load(infile)
infile.close()
ingr_map

,raw_ingr,raw_words,processed,len_proc,replaced,count
id,,,,,,
4308,"medium heads bibb or red leaf lettuce, washed,...",13,"medium heads bibb or red leaf lettuce, washed,...",73,lettuce,4507
4308,mixed baby lettuces and spring greens,6,mixed baby lettuces and spring green,36,lettuce,4507
4308,romaine lettuce leaf,3,romaine lettuce leaf,20,lettuce,4507
4308,iceberg lettuce leaf,3,iceberg lettuce leaf,20,lettuce,4507
4308,red romaine lettuce,3,red romaine lettuce,19,lettuce,4507
...,...,...,...,...,...,...
6702,soybeans,1,soybean,7,soybean,31
3318,goose,1,goose,5,goose,8
47,ajwain,1,ajwain,6,ajwain,13


In [29]:
recipe_ingr_pairs_m1 = (recipes['ingredients'].loc[recipes['submitted'] <= m1]                                          # prior to m1 only,
                        .str.extractall(r"'([^,]*)',").reset_index(level=1, drop=True).rename(columns={0:'raw_ingr'}))  # extract raw ingrs from 'ingredients' col 

ingr_ids = ingr_map['raw_ingr'].reset_index().set_index('raw_ingr')  # get map from raw ingredient to ingr_id

recipe_ingr_pairs_m1 = pd.merge(recipe_ingr_pairs_m1,ingr_ids,how='left',left_on='raw_ingr',right_index=True) # merge raw ingrs with map

recipe_ingr_pairs_m1 = recipe_ingr_pairs_m1.drop(columns='raw_ingr').rename(columns={'id':'ingr_id'}).reset_index().rename(columns={'id':'recipe_id'}) #drop raw ingr string data

recipe_ingr_pairs_m1 = recipe_ingr_pairs_m1.loc[~recipe_ingr_pairs_m1['ingr_id'].isna()] # drop unparsed data (not very many such entries found during exploration)

recipe_ingr_pairs_m1['ingr_id'] = recipe_ingr_pairs_m1['ingr_id'].astype(int)

recipe_ingr_pairs_m1

,recipe_id,ingr_id
0,137739,7933
1,137739,4694
2,137739,4795
3,137739,3723
4,137739,840
...,...,...
1845163,298509,6473
1845164,298509,2499
1845165,298509,2683
1845166,298509,1925


In [31]:
ingr_recipe_count_m1 = recipe_ingr_pairs_m1.groupby('ingr_id').count()  # for each ingredient, count number of recipes in which it appears
ingr_recipe_count_m1.index = ingr_recipe_count_m1.index.astype('int')   #  as an integer not a float
ingr_recipe_count_m1.rename(columns={'recipe_id':'recipe_count'},inplace=True)

recipe_sig_ingr_pairs_m1 = pd.merge(recipe_ingr_pairs_m1,ingr_recipe_count_m1,how='left',left_on='ingr_id',right_index=True)  # merge in the recipe count data
recipe_sig_ingr_pairs_m1 = recipe_sig_ingr_pairs_m1[recipe_sig_ingr_pairs_m1['recipe_count'] >= 50]  # drop ingredients which appear in fewer than 50 recipes

num_recipes_m1 = len(recipe_ingr_pairs_m1['recipe_id'].unique())  # get total number of recipes at m1
recipe_sig_ingr_pairs_m1['ingr_rareness'] = recipe_sig_ingr_pairs_m1['recipe_count'] / num_recipes_m1  # compute ingr rareness ratio for each ingr

recipe_sig_ingr_pairs_m1

,recipe_id,ingr_id,recipe_count,ingr_rareness
0,137739,7933,60,0.000260
2,137739,4795,201,0.000872
3,137739,3723,8773,0.038051
4,137739,840,50255,0.217970
5,137739,5006,37993,0.164786
...,...,...,...,...
1845163,298509,6473,2604,0.011294
1845164,298509,2499,48178,0.208962
1845165,298509,2683,25148,0.109074
1845166,298509,1925,935,0.004055


In [32]:
ingr_rareness_sums_m1 = recipe_sig_ingr_pairs_m1.groupby('recipe_id').agg({'ingr_rareness':'sum'}).rename(columns={'ingr_rareness':'ingr_rareness_sum'})

ingr_rareness_info_m1 = pd.merge(df_train['n_ingredients'],ingr_rareness_sums_m1,how='left',left_index=True,right_index=True)
ingr_rareness_info_m1['innovation_IDF'] = ingr_rareness_info_m1['ingr_rareness_sum'] / ingr_rareness_info_m1['n_ingredients']
ingr_rareness_info_m1

,n_ingredients,ingr_rareness_sum,innovation_IDF
recipe_id,,,
38,4,0.032226,0.008057
39,26,1.147168,0.044122
40,6,0.206125,0.034354
41,15,0.544086,0.036272
43,15,1.101024,0.073402
...,...,...,...
520245,6,0.638990,0.106498
520248,7,0.139205,0.019886
520250,12,0.557536,0.046461


In [33]:
df_train = pd.merge(df_train,ingr_rareness_info_m1['innovation_IDF'],how='left',left_index=True,right_index=True)
df_train['innovation_IDF'].fillna(0,inplace=True)
df_train

,target,total_ratings_curr,age_in_days,rating_pace,RUSNB_proba_pred,mean_rating_curr,minutes,n_steps,n_ingredients,calories,LIX_score,innovation_IDF
recipe_id,,,,,,,,,,,,
38,0,4,5624,0.000711,1.190284e-01,4.250000,1485,13,4,170.9,28.360140,0.008057
39,0,1,5604,0.000178,1.485670e-02,3.000000,265,17,26,1110.7,33.459893,0.044122
40,0,9,5597,0.001608,5.000000e-01,4.333333,35,8,6,311.1,15.944444,0.034354
41,0,2,5599,0.000357,4.268798e-03,4.500000,1460,18,15,536.1,31.488889,0.036272
43,0,1,5612,0.000178,3.387240e-08,1.000000,140,35,15,437.9,28.777387,0.073402
...,...,...,...,...,...,...,...,...,...,...,...,...
520245,0,0,3,0.000000,1.303901e-08,0.000000,35,10,6,206.6,31.076623,0.106498
520248,0,1,3,0.333333,1.055873e-01,5.000000,15,9,7,473.7,30.761905,0.019886
520250,0,1,3,0.333333,3.360625e-06,5.000000,30,11,12,251.5,31.080946,0.046461


In [34]:
recipe_ingr_pairs_t1 = (recipes['ingredients'].loc[recipes['submitted'] <= t1]                                          # prior to t1 only,
                        .str.extractall(r"'([^,]*)',").reset_index(level=1, drop=True).rename(columns={0:'raw_ingr'}))  # extract raw ingrs from 'ingredients' col 

ingr_ids = ingr_map['raw_ingr'].reset_index().set_index('raw_ingr')  # get map from raw ingredient to ingr_id

recipe_ingr_pairs_t1 = pd.merge(recipe_ingr_pairs_t1,ingr_ids,how='left',left_on='raw_ingr',right_index=True) # merge raw ingrs with map

recipe_ingr_pairs_t1 = recipe_ingr_pairs_t1.drop(columns='raw_ingr').rename(columns={'id':'ingr_id'}).reset_index().rename(columns={'id':'recipe_id'}) #drop raw ingr string data

recipe_ingr_pairs_t1 = recipe_ingr_pairs_t1.loc[~recipe_ingr_pairs_t1['ingr_id'].isna()] # drop unparsed data (not very many such entries found during exploration)

recipe_ingr_pairs_t1['ingr_id'] = recipe_ingr_pairs_t1['ingr_id'].astype(int)

recipe_ingr_pairs_t1

,recipe_id,ingr_id
0,137739,7933
1,137739,4694
2,137739,4795
3,137739,3723
4,137739,840
...,...,...
1849799,298509,6473
1849800,298509,2499
1849801,298509,2683
1849802,298509,1925


In [36]:
ingr_recipe_count_t1 = recipe_ingr_pairs_t1.groupby('ingr_id').count()  # for each ingredient, count number of recipes in which it appears
ingr_recipe_count_t1.index = ingr_recipe_count_t1.index.astype('int')   #  as an integer not a float
ingr_recipe_count_t1.rename(columns={'recipe_id':'recipe_count'},inplace=True)

recipe_sig_ingr_pairs_t1 = pd.merge(recipe_ingr_pairs_t1,ingr_recipe_count_t1,how='left',left_on='ingr_id',right_index=True)  # merge in the recipe count data
recipe_sig_ingr_pairs_t1 = recipe_sig_ingr_pairs_t1[recipe_sig_ingr_pairs_t1['recipe_count'] >= 50]  # drop ingredients which appear in fewer than 50 recipes

num_recipes_t1 = len(recipe_ingr_pairs_t1['recipe_id'].unique())  # get total number of recipes at t1
recipe_sig_ingr_pairs_t1['ingr_rareness'] = recipe_sig_ingr_pairs_t1['recipe_count'] / num_recipes_t1  # compute ingr rareness ratio for each ingr

recipe_sig_ingr_pairs_t1

,recipe_id,ingr_id,recipe_count,ingr_rareness
0,137739,7933,60,0.000260
2,137739,4795,201,0.000870
3,137739,3723,8796,0.038067
4,137739,840,50341,0.217862
5,137739,5006,38090,0.164843
...,...,...,...,...
1849799,298509,6473,2605,0.011274
1849800,298509,2499,48296,0.209012
1849801,298509,2683,25187,0.109003
1849802,298509,1925,938,0.004059


In [37]:
ingr_rareness_sums_t1 = recipe_sig_ingr_pairs_t1.groupby('recipe_id').agg({'ingr_rareness':'sum'}).rename(columns={'ingr_rareness':'ingr_rareness_sum'})

ingr_rareness_info_t1 = pd.merge(df_test['n_ingredients'],ingr_rareness_sums_t1,how='left',left_index=True,right_index=True)
ingr_rareness_info_t1['innovation_IDF'] = ingr_rareness_info_t1['ingr_rareness_sum'] / ingr_rareness_info_t1['n_ingredients']
ingr_rareness_info_t1

,n_ingredients,ingr_rareness_sum,innovation_IDF
recipe_id,,,
38,4,0.032289,0.008072
39,26,1.147117,0.044120
40,6,0.206056,0.034343
41,15,0.544320,0.036288
43,15,1.101022,0.073401
...,...,...,...
529564,9,0.732204,0.081356
529568,12,0.478660,0.039888
529577,5,0.447323,0.089465


In [38]:
df_test = pd.merge(df_test,ingr_rareness_info_t1['innovation_IDF'],how='left',left_index=True,right_index=True)
df_test['innovation_IDF'].fillna(0,inplace=True)
df_test

,target,total_ratings_curr,age_in_days,rating_pace,RUSNB_proba_pred,mean_rating_curr,minutes,n_steps,n_ingredients,calories,LIX_score,innovation_IDF
recipe_id,,,,,,,,,,,,
38,0,4,6354,0.000630,1.190284e-01,4.250000,1485,13,4,170.9,28.360140,0.008072
39,0,1,6334,0.000158,1.485670e-02,3.000000,265,17,26,1110.7,33.459893,0.044120
40,0,9,6327,0.001422,5.000000e-01,4.333333,35,8,6,311.1,15.944444,0.034343
41,0,2,6329,0.000316,4.268798e-03,4.500000,1460,18,15,536.1,31.488889,0.036288
43,0,1,6342,0.000158,3.387240e-08,1.000000,140,35,15,437.9,28.777387,0.073401
...,...,...,...,...,...,...,...,...,...,...,...,...
529564,0,0,11,0.000000,1.303901e-08,0.000000,55,8,9,908.9,30.330645,0.081356
529568,0,0,11,0.000000,1.303901e-08,0.000000,35,21,12,358.1,26.595238,0.039888
529577,0,1,10,0.100000,2.218845e-06,0.000000,25,12,5,132.5,33.365248,0.089465


<h2>12. Recipe Innovation Jaccard</h2>

<h2>13. Average Recipe Innovation Jaccard</h2>

These last two features have the potential to require an analysis of a fairly large network (~250 thousand nodes, ~2 million edges) which can be quite computationally intensive, or impossible.  The network in question is a bipartite graph consisting of all recipes and all ingredients as the nodes, with edges connecting a recipe to an ingredient whenever it contains that ingredient.

In order to trim the size of the computation, we examine only two relatively small subsets of the nodes:

popular_recipes = {all recipes which have received at least 50 ratings}
candidate_recipes = {all recipes being considered by the model, for which the RUS NB classifier predicts a target probability of > 0.49}

The set popular_recipes has fewer than 2000 recipes.  The set candidate_recipes varies in size depending on choices of parameters in the model, but in exploratory data analysis it often came out to around ~50000 recipes.

Given a recipe r contained in candidate_recipes, its Jaccard index with a recipe p in popular_recipes is given by

J(r, p) = (# of ingredients contained in both r and p) / (# of ingredientss contained in either r or p)

The recipes r and p will have a high Jaccard index if they have mostly the same ingredients; and a low Jaccard index if they have mostly different ingredients.

Then features 12 and 13 are computed as follows:

jaccard_innovation(r) = 1 - max{ J(r, p) : p in popular_recipes}

jaccard_innovation_avg(r) = 1 - mean{ J(r, p) : p in popular_recipes}

A recipe with high Jaccard innovation scores (close to 1) is very unique in the sense that its ingredients differ significantly from any pre-existing popular recipe.

For recipes in the feature sets which are NOT considered candidate_recipes, we assign them scores of 0 for computational simplicity.  For all intents and purposes, the RUS NB classifier has already eliminated them from our overall model.

Because these features are expensive to compute (over an hour typically on one quad-core CPU, for both train and test), for convenience we have some commented some code which saves the work we've done previously, and reloads it later.

In [39]:
#project.save_data('features_train_partial.csv', df_train.to_csv(), set_project_asset=True, overwrite=True)
#project.save_data('features_test_partial.csv', df_test.to_csv(), set_project_asset=True, overwrite=True)

{'file_name': 'features_test_partial.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'foodcomrecipepopularitypredictor-donotdelete-pr-wiqbzxzcs7ksbf',
 'asset_id': '1c31878c-6c0f-458c-9ca9-64144c949c6a'}

In [48]:
#df_train = pd.read_csv(project.get_file('features_train_partial.csv'),index_col=0)
#df_test = pd.read_csv(project.get_file('features_test_partial.csv'),index_col=0)

In [41]:
edges_m1 = recipe_sig_ingr_pairs_m1[['recipe_id','ingr_id']]  # set edge list for network
edges_m1['ingr_id'] = edges_m1['ingr_id'].astype('float') / 10000   # code ingr_id as decimals < 1 (to distinguish from recipe_id as integer data)
edges_m1

,recipe_id,ingr_id
0,137739,0.7933
2,137739,0.4795
3,137739,0.3723
4,137739,0.0840
5,137739,0.5006
...,...,...
1845163,298509,0.6473
1845164,298509,0.2499
1845165,298509,0.2683
1845166,298509,0.1925


In [43]:
G_m1 = nx.from_pandas_edgelist(edges_m1,'recipe_id','ingr_id')
nx.is_bipartite(G_m1)

True

In [45]:
popular_recipes_at_m1 = recipe_ratings_count[recipe_ratings_count['total_ratings_at_m1'] >= 50].index.to_list()
popular_recipes_at_m1 = [r for r in popular_recipes_at_m1 if r in G_m1.nodes()]

candidate_recipes_train = df_train[df_train['RUSNB_proba_pred'] > .49].index.to_list()
candidate_recipes_train = [r for r in candidate_recipes_train if r in G_m1.nodes()]

len(popular_recipes_at_m1), len(candidate_recipes_train)

(1868, 55163)

In [49]:
#Script below make take about an hour!
"""
df_train['innovation_jaccard'] = [None for i in df_train.index]
df_train['innovation_jaccard_avg'] = [None for i in df_train.index]

for r in candidate_recipes_train:
    r_duplicates = np.array([r for i in popular_recipes_at_m1])
    jaccard_coeffs = [p for u,v,p in nx.jaccard_coefficient(G_m1,zip(r_duplicates,popular_recipes_at_m1))]
    df_train['innovation_jaccard'].loc[r] = 1 - max(jaccard_coeffs)
    df_train['innovation_jaccard_avg'].loc[r] = 1 - np.mean(jaccard_coeffs)

df_train['innovation_jaccard'].fillna(0,inplace=True)
df_train['innovation_jaccard_avg'].fillna(0,inplace=True)

project.save_data('features_train.csv', df_train.to_csv(), set_project_asset=True, overwrite=True)
"""
df_train = pd.read_csv(project.get_file('features_train.csv'),index_col=0)

df_train

,target,total_ratings_curr,age_in_days,rating_pace,RUSNB_proba_pred,mean_rating_curr,minutes,n_steps,n_ingredients,calories,LIX_score,innovation_IDF,innovation_jaccard,innovation_jaccard_avg
recipe_id,,,,,,,,,,,,,,
38,0,4,5624,0.000711,1.190284e-01,4.250000,1485,13,4,170.9,28.360140,0.008057,0.0,0.000000
39,0,1,5604,0.000178,1.485670e-02,3.000000,265,17,26,1110.7,33.459893,0.044122,0.0,0.000000
40,0,9,5597,0.001608,5.000000e-01,4.333333,35,8,6,311.1,15.944444,0.034354,0.5,0.975533
41,0,2,5599,0.000357,4.268798e-03,4.500000,1460,18,15,536.1,31.488889,0.036272,0.0,0.000000
43,0,1,5612,0.000178,3.387240e-08,1.000000,140,35,15,437.9,28.777387,0.073402,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
520245,0,0,3,0.000000,1.303901e-08,0.000000,35,10,6,206.6,31.076623,0.106498,0.0,0.000000
520248,0,1,3,0.333333,1.055873e-01,5.000000,15,9,7,473.7,30.761905,0.019886,0.0,0.000000
520250,0,1,3,0.333333,3.360625e-06,5.000000,30,11,12,251.5,31.080946,0.046461,0.0,0.000000


In [50]:
edges_t1 = recipe_sig_ingr_pairs_t1[['recipe_id','ingr_id']]  # set edge list for network
edges_t1['ingr_id'] = edges_t1['ingr_id'].astype('float') / 10000   # code ingr_id as decimals < 1 (to distinguish from recipe_id as integer data)
edges_t1

,recipe_id,ingr_id
0,137739,0.7933
2,137739,0.4795
3,137739,0.3723
4,137739,0.0840
5,137739,0.5006
...,...,...
1849799,298509,0.6473
1849800,298509,0.2499
1849801,298509,0.2683
1849802,298509,0.1925


In [51]:
G_t1 = nx.from_pandas_edgelist(edges_t1,'recipe_id','ingr_id')
nx.is_bipartite(G_t1)

True

In [52]:
popular_recipes_at_t1 = recipe_ratings_count[recipe_ratings_count['total_ratings_at_t1'] >= 50].index.to_list()
popular_recipes_at_t1 = [r for r in popular_recipes_at_t1 if r in G_t1.nodes()]

candidate_recipes_test = df_test[df_test['RUSNB_proba_pred'] > .49].index.to_list()
candidate_recipes_test = [r for r in candidate_recipes_test if r in G_t1.nodes()]

len(popular_recipes_at_t1), len(candidate_recipes_test)

(1983, 56765)

In [4]:
#Script below may take about an hour!
"""
df_test['innovation_jaccard'] = [None for i in df_test.index]
df_test['innovation_jaccard_avg'] = [None for i in df_test.index]

for r in candidate_recipes_test:
    r_duplicates = np.array([r for i in popular_recipes_at_t1])
    jaccard_coeffs = [p for u,v,p in nx.jaccard_coefficient(G_t1,zip(r_duplicates,popular_recipes_at_t1))]
    df_test['innovation_jaccard'].loc[r] = 1 - max(jaccard_coeffs)
    df_test['innovation_jaccard_avg'].loc[r] = 1 - np.mean(jaccard_coeffs)

df_test['innovation_jaccard'].fillna(0,inplace=True)
df_test['innovation_jaccard_avg'].fillna(0,inplace=True)

project.save_data('features_test.csv', df_test.to_csv(), set_project_asset=True, overwrite=True)
"""
df_test = pd.read_csv(project.get_file('features_test.csv'),index_col=0)

df_test

,target,total_ratings_curr,age_in_days,rating_pace,RUSNB_proba_pred,mean_rating_curr,minutes,n_steps,n_ingredients,calories,LIX_score,innovation_IDF,innovation_jaccard,innovation_jaccard_avg
recipe_id,,,,,,,,,,,,,,
38,0,4,6354,0.000630,1.190284e-01,4.250000,1485,13,4,170.9,28.360140,0.008072,0.0,0.000000
39,0,1,6334,0.000158,1.485670e-02,3.000000,265,17,26,1110.7,33.459893,0.044120,0.0,0.000000
40,0,9,6327,0.001422,5.000000e-01,4.333333,35,8,6,311.1,15.944444,0.034343,0.5,0.975497
41,0,2,6329,0.000316,4.268798e-03,4.500000,1460,18,15,536.1,31.488889,0.036288,0.0,0.000000
43,0,1,6342,0.000158,3.387240e-08,1.000000,140,35,15,437.9,28.777387,0.073401,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
529564,0,0,11,0.000000,1.303901e-08,0.000000,55,8,9,908.9,30.330645,0.081356,0.0,0.000000
529568,0,0,11,0.000000,1.303901e-08,0.000000,35,21,12,358.1,26.595238,0.039888,0.0,0.000000
529577,0,1,10,0.100000,2.218845e-06,0.000000,25,12,5,132.5,33.365248,0.089465,0.0,0.000000
